In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
rawContests = pd.read_csv('data/WorkingData.csv').set_index('ContestId')
contests = pd.read_csv('data/SCALEDEVERYTHING.csv').set_index('ContestId')
contests = contests.drop(columns = ['0HoursOut_scaled', '1HoursOut_scaled', '2HoursOut_scaled', '3HoursOut_scaled'])
expensiveContests = rawContests.loc[rawContests['TotalPrizeAmount']> 100000][[]]
contests.tail(1)

,Success,SportName_NFL,SportName_PGA,SportName_SOCC,SportName_MLB,SportName_NAS,SportName_CFB,SportName_MMA,SportName_LOL,SportName_NHL,...,20HoursOut_scaled,24HoursOut_scaled,36HoursOut_scaled,48HoursOut_scaled,4HoursOut_scaled,5HoursOut_scaled,6HoursOut_scaled,7HoursOut_scaled,8HoursOut_scaled,9HoursOut_scaled
ContestId,,,,,,,,,,,,,,,,,,,,,
55971466,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
numFail = len(contests.loc[contests['Success']==0])
percent = numFail/len(contests)
1-percent

0.9467517361705039

In [4]:
CLF_contests = contests.loc[:].dropna()

In [5]:
CLF_contests['USE_FOR_BALANCE'] = np.random.uniform(0, 1, len(CLF_contests)) <= percent
CLF_contests = CLF_contests.loc[(CLF_contests["USE_FOR_BALANCE"] == True) | (CLF_contests["Success"] == 0)]
CLF_Features = CLF_contests.columns

In [6]:
CLF_Features= CLF_contests.columns.drop('Success')
CLF_Features = CLF_Features.drop('USE_FOR_BALANCE')

In [7]:
# for c in CLF_contests.columns:
#     print(c)

In [8]:
CLF_contests.tail(1)

,Success,SportName_NFL,SportName_PGA,SportName_SOCC,SportName_MLB,SportName_NAS,SportName_CFB,SportName_MMA,SportName_LOL,SportName_NHL,...,24HoursOut_scaled,36HoursOut_scaled,48HoursOut_scaled,4HoursOut_scaled,5HoursOut_scaled,6HoursOut_scaled,7HoursOut_scaled,8HoursOut_scaled,9HoursOut_scaled,USE_FOR_BALANCE
ContestId,,,,,,,,,,,,,,,,,,,,,
55969390,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [9]:
feature_names = CLF_contests.columns

In [10]:
CLF_contests['is_train'] = np.random.uniform(0, 1, len(CLF_contests)) <= .75
train, test = CLF_contests[CLF_contests['is_train']==True], CLF_contests[CLF_contests['is_train']==False]
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 4556
Number of observations in the test data: 1515


### Limit Test data to Expesnive

In [11]:
# test = test.loc[test.index.isin(expensiveContests.index)].dropna()

## ....

In [12]:
actual = test['Success']
y = train['Success']

In [13]:
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(train[CLF_Features], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [14]:
# Create actual english names for the plants for each predicted plant class
preds = clf.predict(test[CLF_Features])
trueIds = test[['Success']]


# View the PREDICTED species for the first five observations
print('Predicted price of first five observations', preds[0:5])
# View the ACTUAL species for the first five observations
print('Actual Price of first five observations', test['Success'].head())

Predicted price of first five observations [0. 0. 0. 0. 1.]
Actual Price of first five observations ContestId
24567091    0.0
24949429    0.0
24964052    0.0
24932088    0.0
24994822    1.0
Name: Success, dtype: float64


In [15]:
featureImportanceDF = pd.DataFrame(data={"Feature":CLF_Features, "Importance":clf.feature_importances_})
featureImportanceDF = featureImportanceDF.sort_values(by=['Importance'],ascending=False)

In [16]:
# featureImportanceDF.head(50)
featureImportanceDF.to_csv("data/RawForestFeatures.csv")

In [17]:
featureImportanceDF.head()

,Feature,Importance
106,4HoursOut_scaled,0.063301
107,5HoursOut_scaled,0.057610
95,PaidUsersInDraftGroup_Scaled,0.049471
98,SuccessThreshold_Scaled,0.035723
108,6HoursOut_scaled,0.035610


In [18]:
accuracy_score(actual, preds)

0.8006600660066007

In [19]:
confuse = pd.DataFrame({'Predict':preds, 'Real':actual})

In [20]:
successes =  len(test.loc[test['Success']==1.0])
fails = len(test.loc[test['Success']==0.0])

In [26]:
print("Contest Reach Threshold? fails:", fails, "---", "successes:", successes)

Contest Reach Threshold? fails: 774 --- successes: 741


In [22]:
print(confusion_matrix(actual, preds))

[[670 104]
 [198 543]]


In [23]:
# true.columns = ['RealVal']
trueDF = pd.DataFrame(data=actual)
trueDF.columns = ['RealVal']
trueDF = trueDF.assign(PredVal= preds.tolist())


In [24]:
failDF = trueDF.loc[trueDF['RealVal'] != trueDF['PredVal']]
falsePositives = failDF.loc[failDF['PredVal']==1][[]]
falseNegatives = failDF.loc[failDF['PredVal']==0][[]]
falsePositivesDF = pd.merge(falsePositives, rawContests, on='ContestId', how='left')
falseNegativesDF = pd.merge(falseNegatives, rawContests, on='ContestId', how='left')

In [25]:
# falsePositivesDF.to_csv("data/ForJon/RawFalsePositives.csv")
# falseNegativesDF.to_csv("data/ForJon/RawFalseNegatives.csv")